In [1]:
%%HTML
<style> code {margin: 0 2px !important;
  padding: 0 5px !important;
  white-space: nowrap !important;
  border: 1px solid #eaeaea !important;
  background-color: #f8f8f8 !important;
  border-radius: 3px !important;} </style>

In [2]:
%load_ext watermark
%watermark -a 'lorenzo Perozzi' -d -m -v -p pandas,seaborn,sklearn

lorenzo Perozzi 2017-08-22 

CPython 3.5.2
IPython 5.1.0

pandas 0.20.2
seaborn 0.7.1
sklearn 0.19.0

compiler   : GCC 4.4.7 20120313 (Red Hat 4.4.7-1)
system     : Linux
release    : 3.10.0-514.21.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit


In [3]:
from IPython.display import Image
from IPython.core.display import HTML 

# Table of contents
1. [Build a database for machine learning prediciton from rockburst hazard assessment at NRS mine](#Build-a-database for-machine-learning-prediciton-from-rockburst-hazard-assessment at NRS mine)
2. [Reading rockbursts, seismic events and tunnel data](#Reading-rockbursts,-seismic-events-and-tunnel-data)
   - [Rockbursts](#1.-Rockbursts)
   - [Tunnel](#2.-Tunnel)
     - [Plotting the tunnel coordinates + rockbursts events](#Plotting-the-tunnel-coordinates-+-rockbursts-events)
   - [Microseismic events](#3.-Microseismic-events)
3. [Match rockbursts with tunnel location](#Match-rockbursts-with-tunnel-location)
   - [Plot projected and original XYZ of rockbursts](#Plot-projected-and-original-XYZ-of-rockbursts)
   - [Plot tunnel + rockbursts + microseismic for visualization](#Plot-tunnel-+-rockbursts-+-microseismic-for-visualization)
4. [Build database for machine learning](#Build-database-for-machine-learning)

# Build a database for machine learning prediciton from rockburst hazard assessment at NRS mine

The aim of this jupyter notebook is to details all essential steps to build a database and compute feature engineering starting from a rockburst database (2013-2016), a tunnel excavation date database (2008-2016) and a microseismic events database from (2009-2016). The final database comprises "snapshot" of the mine state (based on the age of the excavation) at each rockburst data events. Since the rockburst database start in 2013, only the microseismic events after 2012 and the snapshot starting from 2012 are considered. 
Several challenges are faced and in particular:

1. the temporal support: 
   * tunnel excavation date: **monthly**
   * rockbursts: **hourly or minutes**
   * microseismic events: **microseconds**
  
   Since we use the tunnel excavation as base datasets, we have to match rockbursts to the excavation date (e.g.: rockbursts are upscaled to monthly support). This means that more than 1 rockburst could occur within the same month, decreasing the variability of rockbursts features (all the rockbursts for the same month have the same features).
   
2. rockburst location does not match tunnel location

3. how to associate microseismic parameters (e.g.: `Es/Ep`, `MoMag`, `SeisMoment`) to the tunnel coordinates? It easy to compute distance from events, or density of events to each point of the tunnels but how to associate their parameters?

4. we end-up with a database extremely unbalanced. Might be necessary to have another strategy to predict the rockburst hazard assessment.

 
![Image of Yaktocat](https://raw.githubusercontent.com/lperozzi/NRS_RockburstHazardAssessment/master/img/database_sketch.png?token=AF6UYVjUOz1uUd8JD6IKjFpd3if0QQlOks5ZoF95wA%3D%3D)

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/lperozzi/NRS_RockburstHazardAssessment/master/img/database_sketch.png?token=AF6UYVjUOz1uUd8JD6IKjFpd3if0QQlOks5ZoF95wA%3D%3D")

In [5]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
import os
import time
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [6]:
import plotly 
plotly.tools.set_credentials_file(username='lperozzi', api_key='wHxRvOwGxiRTW3OyZeqQ')
import plotly.plotly as py
import plotly.graph_objs as go

# Reading rockbursts, seismic events and tunnel data

### 1. Rockbursts

Rockbursts are available from 2003 to 2016, however, since there is a gap between 2008 and 2013, only rockbursts from 2013 to 2016 are considered.

In [7]:
# rockbusrts 2013 -2016
path ='data/'
allBursts = glob.glob(path + "rockbursts/rockbursts*.csv")
allBursts

cols = list(range(0, 16))
header = ['data','time', 'location','dam_N','dam_E','dam_Z','seis_N','seis_E','seis_Z','MomMag','tonnes','mechanism (?)','description','behind seismic or other barricade (Y/N)','shotcrete only (Y/N)','unsupported face area (Y/N)']

df = pd.DataFrame()
list_ = []
for file_ in allBursts:
    df = pd.read_csv(file_,index_col=False, usecols=cols, header=None, skip_blank_lines=True, 
                     skiprows=2, na_values=['n/a'])
    df.dropna(axis=0, how='all', inplace=True)
    list_.append(df)
rb = pd.concat(list_)

# assign header
rb.columns = header

# transform data to datetime format
rb['Data'] = pd.to_datetime((rb['data'] + ' ' + rb['time']), errors='coerce')
rb.drop(['data','time'], axis=1, inplace=True)

The `mechanism` in the source data are a little bit messy:

| Mechanism      | Count |
|:------------  |:-----:|
|`? Fault slip`  | 2 |   
|`? Guessing local fault slip`| 1|  
|`blast?`|  1|       
|`face`|     1|      
|`fall of ground`|  3|      
|`fault slip`|  7|      
|`fault slip/shake`|  1|      
|`fault slip/strain`|  2|      
|`floor heave`| 1 |     
|`pillar`|     1 |     
|`pillar `|   1|
|`pillar ?`|     1|
|`pillar burst`|     10|
|`pillar?`|     1|
|`seismic FoG`|     1|
|`seismic shake`|     2|
|`strain`|     1|
|`strain `|     1|
|`strain ?`|     1|
|`strain ? `|     1|
|`starin burst`|     10|
|**Total** | **50**

we decide to limited our analysis to the following 5 categories:

- `pillar`
- `strain`
- `fault`
- `pillar and strain`
- `pillar and strain and fault`

In [8]:
# All the pillar bursts
rb['mechanism (?)'][rb['mechanism (?)'].str.contains("pillar", case=False) & 
                  ~rb['mechanism (?)'].str.contains("strain", case=False) & 
                  ~rb['mechanism (?)'].str.contains("fault", case=False)] = 'pillar'
# All the strain bursts
rb['mechanism (?)'][~rb['mechanism (?)'].str.contains("pillar", case=False) & 
                  rb['mechanism (?)'].str.contains("strain", case=False) & 
                  ~rb['mechanism (?)'].str.contains("fault", case=False)] = 'strain'
# All the fault
rb['mechanism (?)'][~rb['mechanism (?)'].str.contains("pillar", case=False) & 
                  ~rb['mechanism (?)'].str.contains("strain", case=False) & 
                  rb['mechanism (?)'].str.contains("fault", case=False)] = 'fault'
# pillar + strain
rb['mechanism (?)'][rb['mechanism (?)'].str.contains("pillar", case=False) & 
                  rb['mechanism (?)'].str.contains("strain", case=False) & 
                  ~rb['mechanism (?)'].str.contains("fault", case=False)] = 'pillar and strain'
# pillar + strain
rb['mechanism (?)'][rb['mechanism (?)'].str.contains("pillar", case=False) & 
                  rb['mechanism (?)'].str.contains("strain", case=False) & 
                  rb['mechanism (?)'].str.contains("fault", case=False)] = 'pillar and strain and fault'

# Remove all other mechanism (seismic shake and Fall of ground (FoG) 
rb = rb[(rb['mechanism (?)'] != 'seismic FoG') &
                                      (rb['mechanism (?)'] != 'seismic shake') &
                                      (rb['mechanism (?)'] != 'blast?') &
                                      (rb['mechanism (?)'] != 'face') &
                                      (rb['mechanism (?)'] != 'fault slip/strain') &
                                      (rb['mechanism (?)'] != 'floor heave') &
                                     (rb['mechanism (?)'] != 'fall of ground')]

Insert `seis` location for the last entry.

In [9]:
rb.reset_index(inplace=True)
rb.loc[rb.index == 38, ['seis_N', 'seis_E', 'seis_Z', 'MomMag']] = rb.iloc[37]['seis_N'], rb.iloc[37]['seis_E'],rb.iloc[37]['seis_Z'],rb.iloc[37]['MomMag']

Make a `yearmonth` variables to match tunnel date.

In [10]:
rb['Year'] = pd.DatetimeIndex(rb['Data']).year
rb['Month'] = pd.DatetimeIndex(rb['Data']).month.map("{:02}".format)
rb['yearmonth'] = str(rb['Year']) + str(rb['Month'])

for i in range(len(rb)):
    rb['yearmonth'][i] = str(rb['Year'][i])+str(rb['Month'][i])

rb['yearmonth'] = pd.to_datetime(rb['yearmonth'], format='%Y%m')

# display result
print(rb.info())
rb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 19 columns):
index                                      39 non-null int64
location                                   38 non-null object
dam_N                                      39 non-null float64
dam_E                                      39 non-null float64
dam_Z                                      39 non-null float64
seis_N                                     39 non-null float64
seis_E                                     39 non-null float64
seis_Z                                     39 non-null float64
MomMag                                     39 non-null float64
tonnes                                     39 non-null object
mechanism (?)                              39 non-null object
description                                38 non-null object
behind seismic or other barricade (Y/N)    39 non-null object
shotcrete only (Y/N)                       38 non-null object
unsupported face area (

,index,location,dam_N,dam_E,dam_Z,seis_N,seis_E,seis_Z,MomMag,tonnes,mechanism (?),description,behind seismic or other barricade (Y/N),shotcrete only (Y/N),unsupported face area (Y/N),Data,Year,Month,yearmonth
0,1,1320-192 stope access upper corner,10249.0,10195.0,3040.0,10249.0,10191.0,3040.0,0.38,0.5,pillar,upper corner of rib pillar (192/205) S/C peel,N,Y,N,2013-03-24 07:17:00,2013,03,2013-03-01
1,2,1320-312 lower/mid wall ejected,10266.0,10316.5,3035.0,10256.0,10325.0,3042.0,1.00,7,pillar,lower to mid wall of draw point rib pillar spi...,N,Y,N,2013-04-23 09:53:00,2013,04,2013-04-01
2,3,1320-205/220 draw point pillar upper corner,10237.0,10211.0,3040.0,10236.0,10206.0,3040.0,0.81,0.75,pillar,upper corner shotcrete peeled off,N,Y,N,2013-05-14 18:24:44,2013,05,2013-05-01
3,4,1320-245/260 draw point pillar,10248.0,10249.0,3040.0,10251.0,10248.0,3042.0,1.10,1,pillar,upper corner shotcrete peeled off,Y,Y,N,2013-05-24 06:15:16,2013,05,2013-05-01
4,5,1320-260 access upper corner,10244.0,10253.0,3040.0,10240.0,10253.0,3034.0,0.13,0.5,strain,upper corner shotcrete peeled off,Y,Y,N,2013-05-24 06:15:24,2013,05,2013-05-01


We end-up with 39 rockburst of the 5 classes described above. The next step is to load the tunnel database....

### 2. Tunnel 
This file comprises the `drift_point.vs` with coordinate, width, height, year, yearmonth of the tunnel as well as the computed distances from the main geological units and faults (done in Gocad by Cecilia)

In [11]:
tn = pd.read_csv('data/distance/info_tunnel.csv')

Remove `-99999 yearmonth` data since it is not possible to match to the closest `yearmonth`.

In [12]:
tn = tn[tn['year'] != -99999]
# tn = tn[tn['year'] >= 2010]
tn['year'] = tn['year'].astype(int)
tn['yearmonth'] = pd.to_datetime(tn['yearmonth'], format='%Y%m')

startime = min(tn.yearmonth)
tn['mineAge'] = (tn.yearmonth - startime)

tn['mineAge_int'] = tn['mineAge'].dt.days
tn['month'] = pd.DatetimeIndex(tn['yearmonth']).month
tn = tn.reset_index(drop=True)
print(tn.info())
tn.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4075 entries, 0 to 4074
Data columns (total 27 columns):
X                                         4075 non-null float64
Y                                         4075 non-null float64
Z                                         4075 non-null float64
width                                     4075 non-null float64
height                                    4075 non-null float64
yearmonth                                 4075 non-null datetime64[ns]
year                                      4075 non-null int64
distance                                  4075 non-null float64
direction[0]                              4075 non-null float64
direction[1]                              4075 non-null float64
direction[2]                              4075 non-null float64
dist_faults                               4075 non-null float64
dist_Contact_from_glencore                4075 non-null float64
dist_Contact_from_glencore_part1          4075 non-null 

,X,Y,Z,width,height,yearmonth,year,distance,direction[0],direction[1],...,dist_dikes,dist_fnorbase,dist_footwall3,dist_footwall_from_glencore,dist_fw2,dist_lgbxbase_east,dist_ol_dia,mineAge,mineAge_int,month
4070,7949.727,10205.34,2406.620,5.0,5.3,2016-08-01,2016,2271.74900,2248.27300,-15.342580,...,1528.649000,47.16618,2533.0560,2186.961000,2281.3880,2549.6470,814.2041,2830 days,2830,8
4071,7902.809,10223.49,2399.007,5.0,5.3,2016-08-01,2016,2319.46300,2295.19100,-33.493230,...,1578.387000,34.74543,2581.4200,2235.016000,2329.8890,2596.5840,858.6109,2830 days,2830,8
4072,10300.040,10303.84,2910.473,5.0,5.3,2016-08-01,2016,53.34663,15.95547,-50.841920,...,20.961720,159.48000,517.3666,31.665000,650.0289,321.1144,690.5846,2830 days,2830,8
4073,10236.450,10201.44,2699.005,5.0,5.3,2016-08-01,2016,48.35584,-33.44859,-11.437550,...,2.386482,146.13480,350.9144,9.780900,419.3764,343.3234,573.2371,2830 days,2830,8
4074,10236.370,10192.60,2698.925,5.0,5.3,2016-08-01,2016,47.05770,-33.37151,-2.604828,...,1.009947,139.50920,346.5991,2.591141,413.9464,346.7471,564.6193,2830 days,2830,8


We end-up with around 4000 tunnel coordinates, representing the excavation rate between 2008 and 2016.

### Plotting the tunnel coordinates + rockbursts events

In [13]:
# text = ['Excavation date: '+'{:d}'.format(tn['mineAge_int'][k],tn['mineAge_int'][k]) for k in range(len(tn['mineAge_int']))]  
# tn['text'] = 'Age '+tn['mineAge_int'].astype(str)   
tunnel = go.Scatter3d(
    x=tn.X,
    y=tn.Y,
    z=tn.Z,
    mode='markers',
    marker=dict(
        size=3,
        color=tn.mineAge_int,
        colorscale= 'Bluered',
        opacity=0.4,
        colorbar=dict(
            x=1,
            y=0.5,
            title='Age'),
    ),
    showlegend=False,
)

rockbursts = go.Scatter3d(
    x=rb.dam_E,
    y=rb.dam_N,
    z=rb.dam_Z,
    name='rockbursts',
    mode='markers',
    marker=dict(
        size=10,
        color='yellow',
        opacity=1,
        line=dict(
            color='black',
            width=1),
    ),
)

data = [tunnel, rockbursts]
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.6, y=1.6, z=0.1)
)
layout = go.Layout(
            autosize=False,
            width=900,
            height=900,
            title ='Tunnel and rockbursts \n <span style="font-size:12px">(Age in days where 0 refers to the beginning of excavation)</span> ',
            
            showlegend=True,
            legend=dict(x=.1, y=1),
            scene = dict(
                    camera=camera,
                    xaxis=dict(
                        title='Easting',
                        titlefont=dict(size=18),
                        ),  
                    yaxis=dict(
                        title='Northing',
                        titlefont=dict(size=18),
                        ),
                    zaxis=dict(
                        title='Depth',
                        titlefont=dict(size=18),
                        ),
            )
            )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Tunnel + Rockbursts')

### 3. Microseismic events
Microseismic events are available from 2009 to 2016. Only microseismic events from 2012 are considered.

In [14]:
path = 'data/microseismic_and_blasting_2009_2016/'
# read only 2013-2016 microseismic events
allSeis = [f for f in os.listdir(path) if re.search(r'(12|13|14|15|16).*\.csv$', f)]
df = pd.DataFrame()
list_ = []
for file_ in allSeis:
    df = pd.read_csv(path + '/' + file_, header=0)
    list_.append(df)
mseis = pd.concat(list_)
mseis.drop([col for col in df.columns if "Unnamed" in col], axis=1, inplace=True)
print(mseis.info())
mseis.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660014 entries, 0 to 149007
Data columns (total 24 columns):
Date(y/m/d)    660014 non-null object
Time(H:M:S)    660014 non-null object
MilliSec       660014 non-null int64
Type           660014 non-null object
Northing       660014 non-null float64
Easting        660014 non-null float64
Depth          660014 non-null float64
NN_Err         660014 non-null float64
EE_Err         660014 non-null float64
DD_Err         660014 non-null float64
Ave_Error      660014 non-null float64
Picks          660014 non-null int64
MoMag          660014 non-null float64
SeisMoment     660014 non-null float64
Energy         660014 non-null float64
Es/Ep          660014 non-null float64
SourceRo       660014 non-null float64
AspRadius      660014 non-null float64
StaticSD       660014 non-null float64
AppStress      660014 non-null float64
DyStressD      660014 non-null float64
MaxDispla      660014 non-null float64
PeakVelPar     660014 non-null float64

,Date(y/m/d),Time(H:M:S),MilliSec,Type,Northing,Easting,Depth,NN_Err,EE_Err,DD_Err,...,Energy,Es/Ep,SourceRo,AspRadius,StaticSD,AppStress,DyStressD,MaxDispla,PeakVelPar,PeakAccPar
0,2012/01/02,06:08:03,484,b,10224.82,10323.08,3149.03,5.98,5.71,11.89,...,24080.0,6.87,3.73,0.00,11488493.0,561897.69,0.0,0.0,0.17,0.00
1,2012/01/02,06:08:03,826,b,10221.03,10321.41,3151.00,12.53,8.60,17.21,...,1007.0,2.31,2.14,0.69,7257412.0,239125.25,10711998.0,0.0,0.06,710046.88
2,2012/01/02,06:08:04,278,b,10225.62,10324.15,3148.48,6.03,5.39,10.99,...,1680.0,4.59,2.19,0.91,7550857.5,305376.63,14254370.0,0.0,0.08,648689.31
3,2012/01/02,06:08:04,713,b,10225.74,10322.72,3149.07,6.17,5.85,12.22,...,1219.0,1.81,3.24,0.75,4300123.0,137191.22,10292576.0,0.0,0.05,489181.91
4,2012/01/02,06:08:04,895,b,10217.07,10323.26,3148.58,7.94,8.67,17.55,...,872.4,1.92,1.40,0.98,12630013.0,421676.06,9430311.0,0.0,0.10,484869.75


Create a `yearmonth` column to match tunnel coordinates.

In [15]:
mseis['yearmonth'] = pd.to_datetime(mseis['Date(y/m/d)']) - pd.offsets.MonthBegin(1)
mseis.head()

,Date(y/m/d),Time(H:M:S),MilliSec,Type,Northing,Easting,Depth,NN_Err,EE_Err,DD_Err,...,Es/Ep,SourceRo,AspRadius,StaticSD,AppStress,DyStressD,MaxDispla,PeakVelPar,PeakAccPar,yearmonth
0,2012/01/02,06:08:03,484,b,10224.82,10323.08,3149.03,5.98,5.71,11.89,...,6.87,3.73,0.00,11488493.0,561897.69,0.0,0.0,0.17,0.00,2012-01-01
1,2012/01/02,06:08:03,826,b,10221.03,10321.41,3151.00,12.53,8.60,17.21,...,2.31,2.14,0.69,7257412.0,239125.25,10711998.0,0.0,0.06,710046.88,2012-01-01
2,2012/01/02,06:08:04,278,b,10225.62,10324.15,3148.48,6.03,5.39,10.99,...,4.59,2.19,0.91,7550857.5,305376.63,14254370.0,0.0,0.08,648689.31,2012-01-01
3,2012/01/02,06:08:04,713,b,10225.74,10322.72,3149.07,6.17,5.85,12.22,...,1.81,3.24,0.75,4300123.0,137191.22,10292576.0,0.0,0.05,489181.91,2012-01-01
4,2012/01/02,06:08:04,895,b,10217.07,10323.26,3148.58,7.94,8.67,17.55,...,1.92,1.40,0.98,12630013.0,421676.06,9430311.0,0.0,0.10,484869.75,2012-01-01


Encode `Type` labels with value between 0 and n_classes-1

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(mseis.Type.values)
list(le.classes_)
mseis['Type_encoded'] = le.transform(mseis['Type'])

Computes closest distance between tunnel coordinates and each microseismic event as well as density of different events (e.g.: how many events for each tunnel points in a radius search between 25 and 500 metres).

In [17]:
from sklearn.neighbors import BallTree,KDTree
events_class = list(mseis.Type.unique())
radii = [25, 50, 100, 250, 500]
for event in events_class:
    tmp_ = mseis.iloc[:,[3,4,5,6]][mseis.iloc[:,[3,4,5,6]]['Type'] == event]
    tree = KDTree(tmp_[['Easting','Northing','Depth']], leaf_size=40)              
    dist,indice = tree.query(tn.loc[:,['X','Y','Z']], k=1) 
#     print(dist,in)
    tn['dist_'+event] = dist
#     for i, ind in enumerate(indice):
#     tn['NN_Err_'+event] = mseis.iloc[indice.ravel(),7].values
#     tn['EE_Err_'+event] = mseis.iloc[indice.ravel(),8].values
#     tn['DD_Err_'+event] = mseis.iloc[indice.ravel(),9].values
#     tn['Ave_Error_'+event] = mseis.iloc[indice.ravel(),10].values
#     tn['Picks_'+event] = mseis.iloc[indice.ravel(),11].values
#     tn['MoMag_'+event] = mseis.iloc[indice.ravel(),12].values
#     tn['SeisMoment_'+event] = mseis.iloc[indice.ravel(),13].values
#     tn['Energy_'+event] = mseis.iloc[indice.ravel(),14].values
#     tn['Es/Ep_'+event] = mseis.iloc[indice.ravel(),15].values
#     tn['SourceRo_'+event] = mseis.iloc[indice.ravel(),16].values
#     tn['AspRadius_'+event] = mseis.iloc[indice.ravel(),17].values
#     tn['StaticSD_'+event] = mseis.iloc[indice.ravel(),18].values
#     tn['AppStress_'+event] = mseis.iloc[indice.ravel(),19].values
#     tn['DyStressD_'+event] = mseis.iloc[indice.ravel(),20].values
#     tn['MaxDispla_'+event] = mseis.iloc[indice.ravel(),21].values
#     tn['PeakVelPar_'+event] = mseis.iloc[indice.ravel(),22].values
#     tn['PeakAccPar_'+event] = mseis.iloc[indice.ravel(),23].values
        
    for radius in radii:
        tn['density_'+event+'_'+str(radius)] = tree.query_radius(tn.loc[:,['X','Y','Z']], r=radius, count_only=True)
tn.head()
# tn.describe()

,X,Y,Z,width,height,yearmonth,year,distance,direction[0],direction[1],...,density_u_50,density_u_100,density_u_250,density_u_500,dist_a,density_a_25,density_a_50,density_a_100,density_a_250,density_a_500
0,10323.92,10198.34,3147.422,5.0,5.3,2008-11-01,2008,131.00790,-74.91635,-3.340698,...,1,5,150,222,224.442061,0,0,0,1,3
1,10316.78,10201.70,3062.079,5.0,5.3,2008-11-01,2008,71.60090,-67.78168,-6.695892,...,1,8,175,230,154.959905,0,0,0,1,3
2,10327.36,10195.59,3061.713,5.0,5.3,2008-11-01,2008,81.31545,-78.36075,-0.588059,...,0,8,175,229,163.641612,0,0,0,1,3
3,10231.52,10236.91,3064.678,5.0,5.3,2008-11-01,2008,30.38189,15.47758,-4.912476,...,4,20,179,231,117.806485,0,0,0,1,3
4,10106.07,10043.83,3069.121,8.0,5.3,2008-11-01,2008,210.07110,142.92980,151.171700,...,1,2,25,238,240.666939,0,0,0,1,3


# Match rockbursts with tunnel location
Since rockbursts and tunnel coordinates doesn't match (neither in `dam` and `seis` location), we have to project the rockbursts coordinate to the closest tunnel coordinate.

Rockburts are matched to tunnel location using `yearmonth` variable. the table will have this format:

 - mine snapshot at rockburst #1


| Easting  | Northing    | Depth | ... | yearmonth | tunnel_age (Month) | RockBursts |
| :--------: |:-----------:| :-----:| :----:| :----: | :----------: | :----------: |
| ... | ...	 | ... | ... | ... | ... | 0
| 10323.92 | 10198.34	 | 3147.422 | ... | 2013-01-01 | 2 | 0|
| 10316.78 | 10201.70	 | 3062.079 | ... | 2013-01-01 | 2 | 0|
| 10327.36 | 10195.59	 | 3061.713 | ... | 2013-01-01 | 2 | 0|
| <span style="color:#EE6363"/> 10249.0 | <span style="color:#EE6363"/>10195.0	 | <span style="color:#EE6363"/>3040.0 | <span style="color:#EE6363"/>... | <span style="color:red"/>2013-01-01 | <span style="color:#EE6363"/>2|<span style="color:#EE6363"/>1
| 10364.88 | 10273.96	 | 3064.678 | ... | 2013-02-01 | 1 | 0|
| 10231.52 | 10236.91	 | 3069.121 | ... | 2013-03-01 | 0 | 0|
|  10249.0 | 10195.0	 | 3040.0 | ... | 2013-03-01 | 0|0



- mine snapshot at rockburst #2


| Easting  | Northing    | Depth | ... | yearmonth | tunnel_age (Month) | RockBursts |
| :--------: |:-----------:| :-----:| :----:| :----: | :----------: | :----------: |
| ... | ...	 | ... | ... | ... | ... | 0
| 10323.92 | 10198.34	 | 3147.422 | ... | 2013-01-01 | 6 | 0|
| 10316.78 | 10201.70	 | 3062.079 | ... | 2013-01-01 | 6 | 0|
| 10327.36 | 10195.59	 | 3061.713 | ... | 2013-01-01 | 6 | 0|
| 10364.88 | 10273.96	 | 3064.678 | ... | 2013-02-01 | 5 | 0|
| 10231.52 | 10236.91	 | 3069.121 | ... | 2013-03-01 | 4 | 0|
| 10327.36 | 10195.59	 | 3061.713 | ... | 2013-04-01 | 3 | 0|
| 10364.88 | 10273.96	 | 3064.678 | ... | 2013-04-01 | 3 | 0|
| 10231.52 | 10236.91	 | 3069.121 | ... | 2013-05-01 | 2 | 0|
| <span style="color:#EE6363"/> 10249.0 | <span style="color:#EE6363"/>10195.0	 | <span style="color:#EE6363"/>3040.0 | <span style="color:#EE6363"/>... | <span style="color:red"/>2013-05-01 | <span style="color:#EE6363"/>2|<span style="color:#EE6363"/>1
| 10327.36 | 10195.59	 | 3061.713 | ... | 2013-05-01 | 2 | 0|
| 10364.88 | 10273.96	 | 3064.678 | ... | 2013-06-01 | 1 | 0|
| 10231.52 | 10236.91	 | 3069.121 | ... | 2013-06-01 | 1 | 0|



One issue is that since the excavation minimal rate is months, we can have more than a rockbursts by month (actually, rockbursts are matched at the end of the excavation month. However a rockbursts could occur in the beginning of the month).

Finding index corresponding to the closest coordinate in the tunnel, matching the rockbursts damage location.

In [18]:
# Shoe-horn existing data for entry into KDTree routines
# tn_xyz = numpy.dstack([y_array.ravel(),x_array.ravel()])[0]
tn_xyz = np.dstack([tn['X'].values, tn['Y'].values, tn['Z'].values])[0]
rb_xyz = np.dstack([rb['dam_E'].values, rb['dam_N'].values, rb['dam_Z'].values])[0]
rb_xyz_list = list(rb_xyz)


def do_kdtree(xyz_to_be_matched,list_xyz_to_match):
    mytree = sc.spatial.cKDTree(xyz_to_be_matched)
    dist, indexes = mytree.query(list_xyz_to_match)
    return indexes

# start = time.time()
# %timeit 
idx_coord_to_match = do_kdtree(tn_xyz,rb_xyz_list).tolist()
# end = time.time()
# print ('Completed in: ',end-start)

~~Assign 1 to `tn` coordinates associated to rockbursts (`idx_to_match`) and 0 to the rest~~

Assign corresponding tunnel coordinates to rb database.

In [19]:
for i, ind in enumerate(idx_coord_to_match):
#     print(i,ind)
    rb.loc[i, 'X'] = tn.loc[ind,'X']
    rb.loc[i, 'Y'] = tn.loc[ind,'Y']
    rb.loc[i, 'Z'] = tn.loc[ind,'Z']

### Plot projected and original XYZ of rockbursts

In [20]:
# text = ['Excavation date: '+'{:d}'.format(tn['mineAge_int'][k],tn['mineAge_int'][k]) for k in range(len(tn['mineAge_int']))]  
# tn['text'] = 'Age '+tn['mineAge_int'].astype(str)   
tunnel = go.Scatter3d(
    x=tn.X,
    y=tn.Y,
    z=tn.Z,
    mode='markers',
    marker=dict(
        size=3,
        color='gray',
#         colorscale= 'Bluered',
        opacity=0.8,
    ),
    showlegend=False,
)

rockbursts = go.Scatter3d(
    x=rb.X,
    y=rb.Y,
    z=rb.Z,
    mode='markers',
    name='projected',
    marker=dict(
        size=10,
        color='yellow',
        opacity=0.8,
        line=dict(
            color='black',
            width=1),
    ),

)

rockbursts1 = go.Scatter3d(
    x=rb.dam_E,
    y=rb.dam_N,
    z=rb.dam_Z,
    name='original',
    mode='markers',
    marker=dict(
        size=10,
        color='red',
        opacity=0.8,
        line=dict(
            color='black',
            width=1),
    ),

)

data = [tunnel , rockbursts, rockbursts1]
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2.5, y=0.1, z=0.1)
)

layout = go.Layout(
            autosize=False,
            width=900,
            height=900,
            title ='Rockbursts original vs projected',
            
            showlegend=True,
            legend=dict(x=.1, y=1),
            scene = dict(
                    camera=camera,
                    xaxis=dict(
                        title='Easting',
                        titlefont=dict(size=18),
                        ),  
                    yaxis=dict(
                        title='Northing',
                        titlefont=dict(size=18),
                        ),
                    zaxis=dict(
                        title='Depth',
                        titlefont=dict(size=18),
                        ),
            )
            )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Rockbursts projected vs original')

Find `tn` index corresponding to the date of the rockbursts.

In [21]:
# Find last index of tn corresponding to the same yearmonth
s=list(rb['yearmonth'])
idx_rb_date = []
for a in s:
    idx_rb_date.append(tn[tn['yearmonth']==a].index[-1])
tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4075 entries, 0 to 4074
Data columns (total 57 columns):
X                                         4075 non-null float64
Y                                         4075 non-null float64
Z                                         4075 non-null float64
width                                     4075 non-null float64
height                                    4075 non-null float64
yearmonth                                 4075 non-null datetime64[ns]
year                                      4075 non-null int64
distance                                  4075 non-null float64
direction[0]                              4075 non-null float64
direction[1]                              4075 non-null float64
direction[2]                              4075 non-null float64
dist_faults                               4075 non-null float64
dist_Contact_from_glencore                4075 non-null float64
dist_Contact_from_glencore_part1          4075 non-null 

In [22]:
tn.head()

,X,Y,Z,width,height,yearmonth,year,distance,direction[0],direction[1],...,density_u_50,density_u_100,density_u_250,density_u_500,dist_a,density_a_25,density_a_50,density_a_100,density_a_250,density_a_500
0,10323.92,10198.34,3147.422,5.0,5.3,2008-11-01,2008,131.00790,-74.91635,-3.340698,...,1,5,150,222,224.442061,0,0,0,1,3
1,10316.78,10201.70,3062.079,5.0,5.3,2008-11-01,2008,71.60090,-67.78168,-6.695892,...,1,8,175,230,154.959905,0,0,0,1,3
2,10327.36,10195.59,3061.713,5.0,5.3,2008-11-01,2008,81.31545,-78.36075,-0.588059,...,0,8,175,229,163.641612,0,0,0,1,3
3,10231.52,10236.91,3064.678,5.0,5.3,2008-11-01,2008,30.38189,15.47758,-4.912476,...,4,20,179,231,117.806485,0,0,0,1,3
4,10106.07,10043.83,3069.121,8.0,5.3,2008-11-01,2008,210.07110,142.92980,151.171700,...,1,2,25,238,240.666939,0,0,0,1,3


### Plot tunnel + rockbursts + microseismic for visualization
Select microseismic events and tunnel data `==` 201301, rockburst `==` 2013.

In [23]:
mseis2 = mseis[(pd.DatetimeIndex(mseis['yearmonth']).year == 2013) & (pd.DatetimeIndex(mseis['yearmonth']).month <= 1)]
tn2 =tn[(tn['yearmonth'] <= tn['yearmonth'][3020])]
rb2 = rb[rb['Year'] == 2013]

# tn['yearmonth'][3020]

~~Compute the kde for events~~

In [24]:
# text = ['Excavation date: '+'{:d}'.format(tn['mineAge_int'][k],tn['mineAge_int'][k]) for k in range(len(tn['mineAge_int']))]  
tn2['text'] = 'Age '+ tn2['mineAge_int'].astype(str)   
tunnel = go.Scatter3d(
    x=tn2.X,
    y=tn2.Y,
    z=tn2.Z,
    mode='markers',
    marker=dict(
        size=3,
        color='black',
      line=dict(
            color='black',
            width=1),
        opacity=0.5,
    ),
    showlegend=True,
    name='Tunnel',
#     text= tn2['text'] 
)

# rob = tn[tn['Rockbursts'] == 1]
rb2['text'] = 'Rockburst event: '+ rb2['yearmonth'].astype(str)

rockbursts = go.Scatter3d(
    x=rb2.X,
    y=rb2.Y,
    z=rb2.Z,
    mode='markers',
    marker=dict(
        size=10,
        color='yellow',
        line=dict(
            color='black',
            width=1),
        symbol='circle',
        
        opacity=1
    ),
    text= rb2['text'],name='Rockbursts'
)

# mseis2 = mseis[(pd.DatetimeIndex(mseis['yearmonth']).year == 2012) | (pd.DatetimeIndex(mseis['yearmonth']).year == 2013)]

# ['a', 'b', 'e', 'n', 'u']

micros_b = go.Scatter3d(
    x=mseis2.Easting[mseis2['Type'] == 'b'],
    y=mseis2.Northing[mseis2['Type'] == 'b'],
    z=mseis2.Depth[mseis2['Type'] == 'b'],
    mode='markers',
    marker=dict(
        size=5,
        color='#b2182b',
        symbol='circle',
        opacity=0.3,
        line=dict(
            color='#ad0317',
            width=0.1),
    ),
    name='micros_blasts'
#     text= rb['text'] 
)


micros_a = go.Scatter3d(
    x=mseis2.Easting[mseis2['Type'] == 'a'],
    y=mseis2.Northing[mseis2['Type'] == 'a'],
    z=mseis2.Depth[mseis2['Type'] == 'a'],
    mode='markers',
    marker=dict(
        size=5,
        color='#d1e5f0',
        symbol='circle',
        opacity=0.3,
        line=dict(
            color='#baddef',
            width=0.1),
    ),
    name='micros_a'
#     text= rb['text'] 
)
micros_e = go.Scatter3d(
    x=mseis2.Easting[mseis2['Type'] == 'e'],
    y=mseis2.Northing[mseis2['Type'] == 'e'],
    z=mseis2.Depth[mseis2['Type'] == 'e'],
    mode='markers',
    marker=dict(
        size=5,
        color='#fddbc7',
        symbol='circle',
        opacity=0.3,
        line=dict(
            color='#f9c5a7',
            width=0.1),
    ),
    name='micros_events'
#     text= rb['text'] 
)
micros_n = go.Scatter3d(
    x=mseis2.Easting[mseis2['Type'] == 'n'],
    y=mseis2.Northing[mseis2['Type'] == 'n'],
    z=mseis2.Depth[mseis2['Type'] == 'n'],
    mode='markers',
    marker=dict(
        size=5,
        color='#67a9cf',
        symbol='circle',
        opacity=0.3,
        line=dict(
            color='#58a0c9',
            width=0.1),
    ),
    name='micros_noise'
#     text= rb['text'] 
)
micros_u = go.Scatter3d(
    x=mseis2.Easting[mseis2['Type'] == 'u'],
    y=mseis2.Northing[mseis2['Type'] == 'u'],
    z=mseis2.Depth[mseis2['Type'] == 'u'],
    mode='markers',
    marker=dict(
        size=5,
        color='#2166ac',
        symbol='circle',
        opacity=0.3,
        line=dict(
            color='#0d59a5',
            width=0.1),
    ),
    name='micros_unknown'
#     text= rb['text'] 
)

# data = [tunnel, rockbursts, micros_b, micros_e, micros_n, micros_u]
data = [tunnel, rockbursts, micros_b, micros_u, micros_e, micros_n, micros_a]
# data = [tunnel, rockbursts, micros_b_kde]
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.6, y=1.6, z=0.1)
)

layout = go.Layout(
            autosize=False,
            width=900,
            height=900,
            title ='Tunnel, rockburst and microseismic events ',
            
            showlegend=True,
            legend=dict(x=.1, y=1),
            scene = dict(
                    camera=camera,
                    xaxis=dict(
                        title='Easting',
                        titlefont=dict(size=18),
                        ),  
                    yaxis=dict(
                        title='Northing',
                        titlefont=dict(size=18),
                        ),
                    zaxis=dict(
                        title='Depth',
                        titlefont=dict(size=18),
                        ),
            )
            )

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Tunnel + Rockbursts')

# Build database for machine learning
The database is made of a snapshot of the mine state at each rockbursts time, with variable `mineAge` that increases for each snapshot.

Columns of interest for machine learning

In [25]:
# pd.set_option('max_rows', 150)
tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4075 entries, 0 to 4074
Data columns (total 57 columns):
X                                         4075 non-null float64
Y                                         4075 non-null float64
Z                                         4075 non-null float64
width                                     4075 non-null float64
height                                    4075 non-null float64
yearmonth                                 4075 non-null datetime64[ns]
year                                      4075 non-null int64
distance                                  4075 non-null float64
direction[0]                              4075 non-null float64
direction[1]                              4075 non-null float64
direction[2]                              4075 non-null float64
dist_faults                               4075 non-null float64
dist_Contact_from_glencore                4075 non-null float64
dist_Contact_from_glencore_part1          4075 non-null 

In [26]:
col = list(tn.columns[0:7]) + list(tn.columns[11::])

In [27]:
ml_database = []
for i, ind in enumerate(idx_rb_date):
    if ind == idx_rb_date[i-1]:
#         print(i,ind)
        xrb = rb['X'].iloc[i]
        yrb = rb['Y'].iloc[i]
        zrb = rb['Z'].iloc[i]
        date = rb['yearmonth'].iloc[i]
        for j in range(len(tmp)):
    #         if tmp['X'].iloc[j] == xrb and tmp['Y'].iloc[j] == yrb and tmp['Z'].iloc[j] == zrb and tmp['yearmonth'].iloc[j] == date :
            if tmp['X'].iloc[j] == xrb and tmp['Y'].iloc[j] == yrb and tmp['Z'].iloc[j] == zrb:
                tmp.loc[j,'Rockburst'] = 1
    else:
#         print(i,ind)
        tmp = tn.loc[:ind+1,col]
        tmp['snapshot'] = i
        xrb = rb['X'].iloc[i]
        yrb = rb['Y'].iloc[i]
        zrb = rb['Z'].iloc[i]
        date = rb['yearmonth'].iloc[i]
        for j in range(len(tmp)):
    #         if tmp['X'].iloc[j] == xrb and tmp['Y'].iloc[j] == yrb and tmp['Z'].iloc[j] == zrb and tmp['yearmonth'].iloc[j] == date :
            if tmp['X'].iloc[j] == xrb and tmp['Y'].iloc[j] == yrb and tmp['Z'].iloc[j] == zrb:
                tmp.loc[j,'Rockburst'] = 1

        ml_database.append(tmp)
ml_database = pd.concat(ml_database)  
ml_database.replace(to_replace=np.NaN,value=0,inplace=True)
ml_database['Rockburst'] = ml_database.Rockburst.astype(int)
# using label encoder to encode snapshot from 0 to len(snapshot) -1
le = preprocessing.LabelEncoder()
le.fit(ml_database['snapshot'])
ml_database['snapshot'] = le.transform(ml_database['snapshot'])
# rest index of ml_database
ml_database = ml_database.reset_index(drop=True)

In [28]:
ml_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69386 entries, 0 to 69385
Data columns (total 55 columns):
Rockburst                                 69386 non-null int64
X                                         69386 non-null float64
Y                                         69386 non-null float64
Z                                         69386 non-null float64
density_a_100                             69386 non-null int64
density_a_25                              69386 non-null int64
density_a_250                             69386 non-null int64
density_a_50                              69386 non-null int64
density_a_500                             69386 non-null int64
density_b_100                             69386 non-null int64
density_b_25                              69386 non-null int64
density_b_250                             69386 non-null int64
density_b_50                              69386 non-null int64
density_b_500                             69386 non-null int64
den

In [29]:
ml_database['yearmonth'] = ml_database['yearmonth'].astype(int) // 10**9
ml_database['mineAge'] = ml_database['mineAge'].astype(int) // 10**9

Save `ml_database` to disk.

In [30]:
ml_database.to_csv('data/dataset_ML/ml_database.csv', index=False)